In [1]:
#default_exp report
%load_ext autoreload
%autoreload 2

# Relatório Consolidado
> Funções de apoio para geração de relatório consolidado dos dados à partir dos arquivos json provenientes da API do [Fiscaliza](https://github.com/ronaldokun/fiscaliza)

In [2]:
#export
import base64
from datetime import datetime
from typing import Union
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from fastcore.foundation import L
from fastcore.xtras import Path
from fiscaliza.info import download_attachments, detalhar_issue

TYPES = {
'category': ('Localidade', 'Stream', 'Irregular', 'Regulatory', 'RiskLevel', 'Type', 'Service', 'Author', 'Node',  
       'Ação', 'Inspeção', 'Relatório', 'DetectionMethod', 'ClassificationMethod', 'occMethod', 'Link', 'Description', 'RelatedFiles', 
       'FileId', 
),
'float64':  ('Frequency', 'Truncated', 'BW', 'minLevel', 'meanLevel', 'maxLevel',
       'meanOCC', 'maxOCC', 'Distance', 'Latitude', 'Longitude', 'Duration(Hours)'),

'string' : ('Identification', 'Station'),

'int32' : ('Samples', 'timeOCC','FreqStart', 'FreqStop'),

'datetime64[ns]' : ('BeginTime', 'EndTime', 'Date'),
}

COLS = TYPES['category'] + TYPES['string'] +  TYPES['float64'][:-1] + TYPES['int32'] + TYPES['datetime64[ns]'] + TYPES['float64'][-1:]

ANEXO = {'FileId': 'id',
         'Author': 'author',
         'Date': 'created_on', 
         'Link': 'content_url',
            }

INFO = {'Inspeção': 'id',
       'Ação': 'id_ACAO',
       'Relatório': 'Relatorio_SEI',
            'Localidade': 'UF_Municipio',
           }


TRANSLATE_COLS = { 'Frequency': 'Frequência (MHz)',
              'Truncated': 'Frequência Truncada (MHz)',
              'BW': 'Largura do Canal (KHz)',
              'meanLevel': 'Nível Médio (dBm | dBμV | dBμV/m)',
              'maxLevel': 'Nível Máximo (dBm | dBμV | dBμV/m)',
              'meanOCC': 'Ocupação Média (%)',
              'maxOCC': 'Ocupação Máxima (%)',
              'Distance': 'Distância (Km)',
              'Duration(Hours)': 'Duração (Horas)', 
              'Samples': 'Número de Amostras',
              'Type': 'Tipo de Emissão',
              'Regulatory': 'Perfil Regulatório',
              'Service': 'Serviço',
              'Irregular': 'Irregularidade',
              'RiskLevel': 'Potencial Lesivo',
              'Author': 'Fiscal',
              'Node': 'Sensor',
              'occMethod': 'Método de Ocupação',
              'DetectionMethod': 'Algoritmo de Detecção',
              'ClassificationMethod': 'Algoritmo de Classificação',
              'Stream': 'Faixa de Frequência (MHz)',
              'BeginTime': 'Início da Monitoração',
              'EndTime': 'Fim da Monitoração',
}

TRANSLATE_REPORT = {
 'Overview': 'Visão Geral',
 'Missing values': 'Valores ausentes',
 'Sample': 'Amostra',
 'Dataset statistics': 'Estatística dos Dados',
 'Dataset': 'Informação',
 'Description': 'Descrição',
 'Author': 'Autor',
 'Alerts': 'Alertas',
 'has a high cardinality': 'tem uma alta cardinalidade',
 ' has ': ' possui ',
 'Reproduction': 'Metadados',
 'Analysis started': 'Análise iniciada',
 'Analysis finished': 'Análise finalizada',
 'Duration': 'Duração',
 'Software version': 'Versão do Software',
 'Download configuration': 'Baixar configuração',
 'second': 'segundo',
 'Real number': 'Número Real',
 '<code>MISSING</code>': '<code>AUSENTES</code>',
 '<code>HIGH CARDINALITY</code>': '<code>ALTA CARDINALIDADE</code>',
 '<small>Date</small>': '<small>Data</small>',
 '>Count</a>': '>Contagem</a>',
 '>Matrix</a>': '>Matriz</a>',
 '<td>Value</td>': '<td>Valor</td>',
 '<td>Count</td>': '<td>Contagem</td>',
 'distinct values': 'valores distintos',
 'High cardinality': 'Alta Cardinalidade',
 'missing values': 'valores ausentes', 
 'Variable types': 'Tipos de Variáveis',
 'Number of variables': 'Número de Variáveis',
 'Number of observations': 'Número de Observações',
 'Missing cells': 'Células Ausentes',
 'Total size in memory': 'Tamanho Total na Memória',
 'Average record size in memory': 'Tamanho Médio de um Registro na Memória',
 'Variables': 'Variáveis',
 'Numeric': 'Numérica',
 'Categorical': 'Categórica',
 'DateTime': 'Data/Hora',
 'Distinct': 'Distintas',
 'Missing': 'Ausentes',
 'Infinite': 'Infinito',
 'Mean': 'Média',
 'Standard deviation': 'Desvio padrão',
 'Minimum 10 values': '10 Valores Mínimos',
 'Maximum 10 values': '10 Valores Máximos',
 'Minimum': 'Mínimo',
 'Maximum': 'Máximo',
 'Negative': 'Negativo',
 'median': 'Mediana',
 'Memory size': 'Tamanho na memória',
 'Toggle details': 'Detalhes',
 'Statistics': 'Estatísticas',
 'Histogram': 'Histograma',
 'Histograma with fixed size bins': 'Histograma com tamanho fixo de <i>bins</i>',
 'Common values': 'Valores comuns',
 '>Common Values<': '>Valores Comuns<',
 'Extreme values': 'Valores extremos',
 'Quantile statistics': 'Estatísticas de Quantis',
 'Descriptive statistics': 'Estatísticas Descritivas',
 'percentile': 'percentil',
 'Range': "Intervalo",
 'Interquartile range': 'Intervalo Interquartil',
 'Coefficient of variation': 'Coeficiente de Variação',
 'Kurtosis': 'Curtose',
 'Median Absolute Deviation': 'Desvio Mediano Absoluto',
 'Skewness': 'Distorção',
 'Sum': 'Soma',
 'Variance': 'Variância',
 'Monotonicity': 'Monotonicidade',
 'Not monotonic': 'Não Monotônico',
 'monotonic': 'Monotônico',
 'Increasing': 'Crescente',
 'Decreasing': 'Decrescente',
 '>Category Frequency Plot<': '>Gráfico de Contagem por Categoria<',
 'Frequency': 'Frequência', 
 '>Categories</a>': '>Categorias</a>',
 'Unique': 'Únicos',
 'The number of unique values (all values that occur exactly once in the dataset).': 'Número de valores únicos (todos os valores que ocorrem apenas uma vez no conjunto de dados).',
 '<th>1st row</th>': '<th>1ª linha</th>',
 '<th>2nd row</th>': '<th>2ª linha</th>',
 '<th>3rd row</th>': '<th>3ª linha</th>',
 '<th>4th row</th>': '<th>4ª linha</th>',
 '<th>5th row</th>': '<th>5ª linha</th>',
 '>First rows</h2>': '>Primeiros registros</h2>',
 '>Last rows</h2>': '>Últimos registros</h2>',
 '>Random sample<': '>Amostra Aleatória<',
 'Nullity matrix is a data-dense display which lets you quickly visually pick out patterns in data completion.': 'A Matriz de Nulidade é uma visualização de dados densa que permite facilmente detectar padrões na disponibilidade dos dados.',
 'A simple visualization of nullity by column.': 'Visualização simples de nulidade por coluna.', 
 "Report generated with" : 'Relatório gerado com a biblioteca <a href="https://github.com/ronaldokun/anatel-report">anatel-report</a> e ',  
}

DROP = ('minLevel', 
        'Station', 
        'Identification', 
        'timeOCC',
        'FreqStart',
        'FreqStop',
        'Latitude',
        'Longitude',
        'Description',
        'Link',
        'Date',
        'FileId',
        'RelatedFiles')

c:\Users\rsilva\Miniconda3\envs\report\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#export
def df_from_json(path):
    j = path.read_json()
    df = pd.DataFrame(j['MeasurementData'])
    
    for k,v in ANEXO.items():
        df[k] = j.get(v, pd.NA)

    inspecao = j.get('Inspecao', {})

    for k,v in INFO.items():
        df[k] = inspecao.get(v, pd.NA)

    df['Identification'] = df['Description']
    df.drop(['Description'], axis=1, inplace=True)
    
    for fluxo in j['ReferenceData1']:
        for col in list(fluxo.keys())[1:]: # skip 'PK*'
            df.loc[df.FK1 == fluxo['PK1'], col] = fluxo[col]
    for fluxo in j['ReferenceData2']:
        df.loc[df.FK2 == fluxo['PK2'], 'occMethod'] = fluxo['occMethod']
    for fluxo in j['ReferenceData3']:
        df.loc[df.FK3 == fluxo['PK3'], 'DetectionMethod'] = fluxo['Detection']
    for fluxo in j['ReferenceData4']:
        df.loc[df.FK4 == fluxo['PK4'], 'ClassificationMethod'] = fluxo['Classification']
    return df.drop(['FK1', 'FK2', 'FK3', 'FK4'], axis=1)

In [4]:
#export
def process_jsons(path: Path, savepath: Union[str, Path] = None) -> pd.DataFrame:
    """
    Generate a pandas data frame from a given set of jsons files in path
    """
    path = Path(path)
    files = L(j for j in path.iterdir() if j.suffix == '.json')
    dfs = []
    older = []

    for f in files:
        try:
            dfs.append(df_from_json(f))
        except Exception:
            older.append(f)

    if not dfs:
        raise ValueError(f'Não foram encontrados arquivos válidos na pasta {path}')

    df = pd.concat(dfs, ignore_index=True).sort_values(by='Truncated').reset_index(drop=True)
    df['Date'] = df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
    df['BeginTime'] =  df.BeginTime.apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))                        
    df['EndTime'] = df.EndTime.apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))
    df['Duration'] = df.EndTime - df.BeginTime
    df['Duration(Hours)'] = (df.Duration.dt.days * 24 + \
                    df.Duration.dt.components['hours'] + \
                    df.Duration.dt.components['minutes'] / 60 + \
                    df.Duration.dt.seconds / 3600).round(2)
    df.drop('Duration', axis=1, inplace=True)
    df.loc[df.Service.isin({-1, '-1'}), 'Service'] = pd.NA
    df.loc[df.Station.isin({-1, '-1'}), 'Station'] = pd.NA
    df.loc[df.Distance.isin({'-', 'Inf'}), 'Distance'] = np.nan
    df.loc[df.RiskLevel.isin({'-'}), 'RiskLevel'] = 'Nulo'
    df['Stream'] = df.FreqStart.astype('int').astype('string') + '-' + df.FreqStop.astype('int').astype('string')
    for dtype, cols in TYPES.items():
        for col in cols:
            df[col] = df[col].astype(dtype)
    if older:
        print(f'Os arquivos seguintes não foram processados por não atender aos padrões: {older}')
    df =  df.loc[:,COLS]    
    if savepath is not None:
        savepath = Path(savepath)
        savepath.mkdir(exist_ok=True, parents=True)
        df.to_excel(savepath / 'report_data.xlsx', index=False)

    df.drop(list(DROP), axis=1, inplace=True)
    return df.rename(columns=TRANSLATE_COLS)    

In [5]:
#export 
def generate_report(df: pd.DataFrame, 
                    title: str = 'Relatório de Dados Consolidados') -> str:
    """
    Generate a report from a given set of jsons files in path
    """
    datadir = Path.cwd().parent / 'dados' / 'html'
    icon = base64.b64encode((datadir / 'Puzzle.ico').read_bytes()).decode('utf-8')
    icon = rf'src="data:image/jpeg;base64,{icon}"'
    profile = ProfileReport(df,
    config_file= datadir / 'report_config.yaml',
    title=title, 
    html={'style': {'logo': 'icon'}},
    pool_size=0)

    html = profile.to_html()
    html = html.replace('src=icon', icon)
    
    for k,v in TRANSLATE_REPORT.items():
        html = html.replace(k, v)
    return html    

In [6]:
def issue2report(
    issue: Union[str, int],
    savepath: Union[str, Path] = None,
    title: str = 'Relatório de Dados Consolidados',
    login: str = None,
    senha: str = None,
    api: str = None,
    teste: bool = True,
):
    savepath = Path(savepath)
    js = savepath / 'json'
    js.mkdir(exist_ok=True, parents=True)
    out = savepath / 'report'
    out.mkdir(exist_ok=True, parents=True)
    datadir = Path.cwd().parent / 'dados' / 'html'
    info = detalhar_issue(issue, login, senha, api, teste=teste)
    #download_attachments(issue, js, login, senha, api, teste)
    js = js / str(issue)
    df = process_jsons(js, savepath)
    n_files = df.Inspeção.unique().shape[0]
    html = generate_report(df, title)
    header = (datadir / 'header.html').read_text()
    html = html.replace('<a class=anchor-pos id=top></a>', f'<a class=anchor-pos id=top></a>{header}')
    subject = info['subject'].lower()
    match subject[:3]:
        case 'ins':
            subject = 'Inspeção'
        case 'aca':
            subject = 'Ação'
        case 'sol':
            subject = 'Solicitação de Inspeção'
        case _:
            subject = 'Tarefa'
    html = html.replace('{IssueType}', subject)
    html = html.replace('{IssueNumber}', info['id'])
    html = html.replace('{ExtractionDate}', datetime.now().strftime('%d/%m/%Y às %H:%M:%S'))
    html = html.replace('{NumberOfFiles}', str(n_files))
    html = html.replace('a{color:#337ab7;text-decoration:none}', 'a{color:#18bc9c;text-decoration:none}')
    (out / f'{issue}_report.html').write_text(html)
    return html   

In [7]:
html = issue2report(77719, Path.cwd(), title='PMEC 2022 - Etapa 1 de 3', login='user_sentinela', api='3a2cb45a5b5ff37d566a6860ba8e96f7f32cff98', teste=False)

In [9]:
html.html

Render HTML: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it]


'<!doctype html><html lang=en><head><meta charset=utf-8><meta name=viewport content="width=device-width, initial-scale=1, shrink-to-fit=no"><meta name=description content="Profile report generated with the `pandas-profiling` Python package"><meta name=author content="Simon Brugman and the open source community."><meta name=generator content="Pandas Profiling v3.2.0"><meta name=url content=https://github.com/pandas-profiling/pandas-profiling><meta name=date content="2022-06-11 20:46:12.733914"><title>PMEC 2022 - Etapa 1 de 3</title><style>\n/*!\n * Bootstrap v3.3.7 (http://getbootstrap.com)\n * Copyright 2011-2016 Twitter, Inc.\n * Licensed under MIT (https://github.com/twbs/bootstrap/blob/master/LICENSE)\n *//*! normalize.css v3.0.3 | MIT License | github.com/necolas/normalize.css */html{font-family:sans-serif;-webkit-text-size-adjust:100%;-ms-text-size-adjust:100%}body{margin:0}article,aside,details,figcaption,figure,footer,header,hgroup,main,menu,nav,section,summary{display:block}aud